In [49]:
import os

In [50]:
def extract_data(root):
    result_dict = {}  # Create an empty dictionary to store the results
    # Walk through all directories and subdirectories
    for folder, subdirs, files in os.walk(root):
        if os.path.basename(folder).startswith("o3cpu"):
            # Check if 'terminaloutput.txt' is not in the list of files in the folder
            if "terminaloutput.txt" not in files:
                # open the "stats.txt" file and read the last line
                with open(os.path.join(folder, "stats.txt")) as f:
                    overallMisses = 0
                    # read all lines into a list
                    lines = f.readlines()
                    for line in lines:
                        if "simSeconds" in line:
                            # extract the simSeconds value from the line
                            simSeconds = float(line.split()[1])
                        if "system.cpu.numCycles" in line:
                            # extract the numCycles value from the line
                            numCycles = int(line.split()[1])
                        if "system.cpu.branchPred.condPredicted" in line:
                            # extract the conditional predicted branches value from the line
                            condPredicted = int(line.split()[1])
                        if "system.cpu.branchPred.condIncorrect" in line:
                            # extract the conditional incorrect branches value from the line
                            condIncorrect = int(line.split()[1])
                        if "system.cpu.commit.branchMispredicts" in line:
                            # extract the branch mispredicts value from the line
                            branchMispredicts = int(line.split()[1])
                        if "overallMisses::total" in line:
                            # extract the overall misses value from the line
                            overallMisses += int(line.split()[1])
                result_dict[os.path.basename(folder)] = {"Seconds": simSeconds, "Cycles": numCycles, "Conditional Predicted": condPredicted,
                                                         "Conditional Incorrect": condIncorrect, "Branch Mispredicts": branchMispredicts,
                                                         "Overall Cache Misses": overallMisses}  # Create an empty dictionary for the folder

    return result_dict

In [52]:
data = extract_data(".")

# print data in a table
print("Benchmark\tSeconds\t\tCycles\t\tConditional Predicted\tConditional Incorrect\tBranch Mispredicts\tOverall Cache Misses")
for key, value in data.items():
    print("{}\t\t{}\t\t{}\t\t{}\t\t\t\t{}\t\t\t\t{}\t\t\t\t{}".format(key.split("-")[2], value["Seconds"], value["Cycles"], value["Conditional Predicted"],
                                                                         value["Conditional Incorrect"], value["Branch Mispredicts"],
                                                                         value["Overall Cache Misses"]))

Benchmark	Seconds		Cycles		Conditional Predicted	Conditional Incorrect	Branch Mispredicts	Overall Cache Misses
o3cpu-spectre-localBP		0.008116		8116491		863387				7684				7302				56884
o3cpu-spectre-multiperspective-perceptron-8KB		0.240178		240177980		22681575				114187				113674				1637081
o3cpu-spectre-ltage		0.567366		567365605		50246568				41038				40579				3898824
o3cpu-spectre-bimodebp		0.008116		8116133		857885				6890				6308				56763
o3cpu-spectre-tournamentBP		0.022431		22430566		2195718				10817				10356				154371
o3cpu-spectre-TAGE		0.008017		8016975		815200				2685				2224				56570
